# XML

## Allgemeines zur Extendable Markup Language (XML)

XML ist eine Markup-Sprache, mit der Informationen strukturiert werden können.
Wesentliches Element sind die charakteristischen XML-Tags in spitzen Klammern.

### Bäume, Tags und Knoten vollen Text

Die Tags treten i.d.R. paarweise auf, auf einen öffnenden Tag (z.B. `<beispieltag>`) folgt ein schließender Tag (`</beispieltag>`).
Tags können verschachtelt werden und ergeben eine Art von Baumstruktur, z.B.:

```xml
<actor>
    <actorID>http://d-nb.info/gnd/118585231</actorID>
    <nameActorSet>
        <appellationValue>Müller, Victor</appellationValue>
    </nameActorSet>
    <vitalDatesActor>
        <earliestDate>1830</earliestDate>
        <latestDate>1871</latestDate>
    </vitalDatesActor>
</actor>
```

In diesem Beispiel werden textuelle Informationen durch tags strukturiert.
Der sog. Wurzelknoten ist `<actor>`.
Er verfügt über mehrere Kind-Knoten (`<actorID>`, `<nameAcotr Set>`, `vitalDatesActor>`).
Knoten können entweder andere Knoten oder Text.
Der Knoten `<appellationValue>` enthält z.B. den Text "Müller, Victor".


### Attribute

XML-Tags können um Attribute ergänzt werden.

```xml
<actor type="http://terminology.lido-schema.org/actor_type/person">
    <actorID type="http://terminology.lido-schema.org/identifier_type/uri" source="http://d-nb.info/" pref="preferred">http://d-nb.info/gnd/118585231</actorID>
    <nameActorSet>
        <appellationValue lang="de">Müller, Victor</appellationValue>
    </nameActorSet>
    <vitalDatesActor>
        <earliestDate type="birthDate">1830</earliestDate>
        <latestDate type="deathDate">1871</latestDate>
    </vitalDatesActor>
</actor>
```

Attribute bestehen aus einem Namen (z.B. `type`) und einem Wert (z.B. `http://terminology.lido-schema.org/actor_type/person`).
Attribute können Tag genauer charakterisieren.
So gibt bspw. das Attribut `lang="de"` an, dass der Text des `<appellationValue>` deutsch ist.

Die erlaubten Attribute werden im jeweiligen XML-Schema beschrieben ([dazu später mehr](#XML-Schema)).

### Namensräume

XML-Dateien arbeiten in aller Regel mit sog. Namensräumen.
In komplexen Dateien oder bei Daten aus unterschieldichen Quellen kann nicht ausgeschlossen werden, dass z.B: ein Tag wie `<actor>` für verschiedene Dinge verwendet wird.
Um durchgehend Eindeutigkeit zu garantieren, werden Tags mit Namensräumen versehen, d.h. ihnen wird ein URI als Präfix vorangestellt.
Elemente aus dem LIDO-Schema erhalten z.B. das Präfix `http://www.lido-schema.org`.
Während `<actor>` potenziell mehrdeutig sein kann, ist `<{http://www.lido-schema.org}actor>` eindeutig als `<actor>` aus dem LIDO-Schema ausgezeichnet.

Die Präfixe werden im Wurzel- oder ein einem Vorfahrenknoten Präfixe definiert.
Bei LIDO geschieht dies für gewöhnlich im `<lidoWrap>`-Element.

```xml
<lido:lidoWrap xmlns:lido="http://www.lido-schema.org">
...
</lido:lidoWrap>
```

Das Präfix `xml:`, das mit dem Namensraum `http://www.w3.org/XML/1998/namespace` verbunden ist und im Beispiel beim Attribut `xml:lang` zu finden ist, steht in einer XML standardmäßig zur Verfügung und braucht nicht eigens deklariert zu werden.


```xml
<lido:actor lido:type="http://terminology.lido-schema.org/actor_type/person">
    <lido:actorID lido:type="http://terminology.lido-schema.org/identifier_type/uri" lido:source="http://d-nb.info/" lido:pref="preferred">http://d-nb.info/gnd/118585231</lido:actorID>
    <lido:nameActorSet>
        <lido:appellationValue xml:lang="de">Müller, Victor</lido:appellationValue>
    </lido:nameActorSet>
    <lido:vitalDatesActor>
        <lido:earliestDate lido:type="birthDate">1830</lido:earliestDate>
        <lido:latestDate lido:type="deathDate">1871</lido:latestDate>
    </lido:vitalDatesActor>
</lido:actor>
```

## XML mit Python parsen

In Python steht zur Arbeit mit XML-Dateien z.B. das [Modul lxml.etree](https://lxml.de/tutorial.html) zur Verfügung.

Wir importieren zunächst `lxml.etree`:

In [ ]:
from lxml import etree

Zum Parsen einer Datei stellt `lxml.etree` die Methode [`.parse()`](https://lxml.de/tutorial.html#the-parse-function) zur Verfügung.

In [ ]:
tree = etree.parse("data/staedel/records.xml")

Die Variable `tree` enthält nur eine geparste Repräsentation der XML-Datei (als [`_ElementTree`-Objekt](https://lxml.de/3.1/api/private/lxml.etree._ElementTree-class.html)).

Ein `_ElementTree`-Objekt verfügt z.B. über die Methode `.iter()`, die Elemente des XML-Baums (z.B. mit einer `for`-Schleife) iterierbar macht.

In [ ]:
for i in tree.iter():
    print(i)

Die einzelnen Elemente (der Klasse `_Element`) haben wiederum einige nützliche Methoden.

- `.tag` gibt den Namen des Knotens aus
- `.attrib` gibt die Attribute des Elements als Dictionary aus

Modifizieren Sie die obige `for`-Schleife und lassen Sie sich z.B. die Knotennamen und Attribute ausgeben.

In [ ]:
for i in tree.iter():
    print(tree.getpath(i))

## XPath

Ehe wir die interessanteren Funktionen von `lxml.etree` nutzen zu können, müssen wir einen Blick auf [XPath](https://lxml.de/xpathxslt.html) werfen.

XPath ist eine Abfragesprache für XML-Dateien, mit der bestimmte Knoten, Knotengruppen und andere Elemente eines XML-Objekts ausgewählt werden können.

XPath stellt zur Beschreibung der Baumstruktur eine Syntax zur Verfügung, die an die Darstellung von Dateipfaden (insbesondere auf UNIX-Systemen) oder auch URLs erinnert.

`_ElementTree` hat eine `.getpath()`-Methode, die den XPAth-Pfad eines Objekts in einem XML-Baum ausgibt.
Der folgende Code-Schnipsel gibt also für jedes Element unseres XML-Objekts `tree` den XPath-Pfad aus, gewissermaßen die Adresse des Elements, das dessen genaue Verortung im Baum ermöglicht.

In [ ]:
for i in tree.iter():
    print(tree.getpath(i))

Mit Hilfe der Adresse kann ein Knoten des Baums ausgewählt werden. Hierfür steht u.a. die Methode `.xpath()` zur Verfügung.
Verwenden wir eine XPath-Adresse in der `.xpath()`-Methode, erhalten wir die eine Liste aller Elemente zurück, auf die die Adresse passt.

Beachten Sie, dass die `.xpath()`-Methode die Namespace-Präfixe des XPath-Pfads nicht kennt, sie müssten eigens (in Form eines Dictionarys) angegeben werden.

In [ ]:
tree.xpath("/*/*[3]/*[36]/*[2]/lido:lidoWrap/lido:lido/lido:descriptiveMetadata/lido:objectIdentificationWrap/lido:repositoryWrap/lido:repositorySet/lido:repositoryLocation/lido:placeID", namespaces = {'lido' : 'http://www.lido-schema.org'})

Da die Adresse auf genau ein Objekt referenziert, enthält die Liste genau ein Objekt.
Löschen wir jedoch z.B. die numerischen Indizes aus dem Pfad, erhöht sich die Treffermenge.

In [ ]:
tree.xpath("/*/*/*/*/lido:lidoWrap/lido:lido/lido:descriptiveMetadata/lido:objectIdentificationWrap/lido:repositoryWrap/lido:repositorySet/lido:repositoryLocation/lido:placeID", namespaces = {'lido' : 'http://www.lido-schema.org'})

Die [XPath-Syntax](https://www.w3schools.com/xml/xpath_syntax.asp) sieht zahlreiche Platzhalter vor.

Einige Beispiele:

- `//tagname`: Wählt alle `tagname`-Knoten an einer beliebigen Position im XML-Baum aus
- `vorfahre//tagname`: Wählt alle `tagname`-Knoten aus, die dem `vorfahre`-Knoten untergeordnet sind
- `//@xml:lang`: gibt Werte aller `xml:lang`-Attribute aus
- `//*[@xml:lang]`: gibt alle Knoten mit einem `xml:lang`-Attribut aus
- `//*[@xml:lang="en"]`: gibt alle Knoten aus, deren `xml:lang`-Attribut den Wert `"en"` hat
- mit `..` geht man einen Schritt nach oben in der XML-Hierarchie. `//lido:appellationValue/..` gibt also die Elemente zurück, die einen `lido:appellationValue`-Knoten enthalten.
- `//tagname/text()`: gibt den Textinhalt des `tagname`-Knotens zurück

Anregungen zum Herumspielen:

1. ...
2. ...

In [ ]:
XPATH = "" # <-hier den XPATH-Ausdruck eingeben
tree.xpath(XPATH, namespaces = {'lido' : 'http://www.lido-schema.org'})